In [1]:
# Install required packages (if not already installed)
!pip install --quiet chromadb transformers torch llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 35.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 39.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:

In [2]:

import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import chromadb
from llama_cpp import Llama

# Config for embedding model and Chroma
MODEL_NAME = 'intfloat/e5-base-v2'
CHROMA_PATH = '/kaggle/input/database-haj/hajj_e5_chroma_backup'
COLLECTION_NAME = 'hajj_e5'
PASSAGE_PREFIX = 'passage: '
QUERY_PREFIX = 'query: '
JSON_PATH = '/kaggle/input/database-haj/hajj_chunks_e5.json'
NPY_PATH = '/kaggle/input/database-haj/hajj_embeddings_e5.npy'

# Path to your quantised LLM file (gguf format)
#LLM_PATH = '/path/to/mistral-7b-instruct-q4_k_m.gguf'  # TODO: replace with actual path on your Pi

# Maximum tokens for generation and context
MAX_TOKENS = 256

# Device selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:

# Load E5 model and tokenizer for query encoding
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

def embed_query(text: str):
    """Encode a query string into an embedding vector using E5 and normalise it."""
    input_text = QUERY_PREFIX + text
    encoded = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        out = model(**encoded)
        token_embeds = out.last_hidden_state
        mask = encoded['attention_mask'].unsqueeze(-1)
        sum_embeds = (token_embeds * mask).sum(dim=1)
        sum_mask = mask.sum(dim=1)
        embed = (sum_embeds / sum_mask).squeeze(0).cpu().numpy()
    norm = np.linalg.norm(embed)
    if norm > 0:
        embed = embed / norm
    return embed


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:

# Copy the folder into a writable location (if it came from a read-only dataset)
import shutil
shutil.copytree('/kaggle/input/database-haj/hajj_e5_chroma_backup', '/kaggle/working/hajj_e5_chroma')

# Then point Chroma at the copy
CHROMA_PATH = '/kaggle/working/hajj_e5_chroma'
client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = client.get_or_create_collection(name='hajj_e5', metadata={'hnsw:space': 'cosine'})

# Helper search function with lexical re-ranking as fallback
def search(query_str: str, top_k: int = 10, re_rank: bool = True):
    query_embed = embed_query(query_str)
    result = collection.query(query_embeddings=[query_embed.tolist()], n_results=top_k)
    ids = result['ids'][0]
    dists = result['distances'][0]
    docs = result['documents'][0]
    metas = result['metadatas'][0]
    hits = []
    for id_, dist, doc, meta in zip(ids, dists, docs, metas):
        hits.append({'id': id_, 'distance': float(dist), 'text': doc, 'metadata': meta})
    if re_rank:
        query_tokens = set(query_str.lower().split())
        for h in hits:
            text_tokens = set(h['text'].lower().split())
            h['lexical_score'] = len(query_tokens & text_tokens)
        hits.sort(key=lambda x: x['lexical_score'], reverse=True)
    return hits


In [5]:

def build_prompt(question: str, sources: list):
    """Construct a prompt for the LLM using the question and retrieved sources."""
    prompt_lines = []
    prompt_lines.append("You are an assistant answering questions about Hajj and Umrah.")
    prompt_lines.append("Answer concisely in plain English so that the response can be read aloud.")
    prompt_lines.append("Keep the answer to no more than 3–4 sentences.")
    prompt_lines.append(f"Question: {question}")
    prompt_lines.append("Sources:")
    for i, src in enumerate(sources, 1):
        text = src['text'].replace("", " ").strip()
        if len(text) > 300:
            text = text[:297] + '...'
        prompt_lines.append(f"[{i}] {text}")
    prompt_lines.append("Answer:")
    return "".join(prompt_lines)


In [6]:
!pip -q install llama-cpp-python huggingface_hub

In [17]:
from openai import OpenAI
client = OpenAI(api_key="")

In [18]:
def generate_answer(question: str, top_k: int = 5):
    hits = search(question, top_k=top_k, re_rank=True)
    prompt = build_prompt(question, hits)
    result = client.chat.completions.create(
        model="gpt-4o-mini",   # or "gpt-4o", "gpt-4-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.2,
        max_tokens=MAX_TOKENS,
        stop=["Sources:", "Question:"]
    )
    answer = result.choices[0].message.content.strip()
    return answer, hits

In [21]:

# Example question
question = "from where tawaf starts?"
answer, sources = generate_answer(question, top_k=5)
print("Answer:", answer)


Answer: Tawaf starts from the Black Stone, known as Al-Hajar Al-Aswad, which is located at one corner of the Kaaba. Pilgrims begin their circumambulation by facing the Black Stone and ideally touching or pointing to it. They then walk around the Kaaba seven times in a counterclockwise direction.


In [22]:
print("Sources used:")

for i, src in enumerate(sources, 1):
    print(f"[{i}] {src['text'][:150].replace('','')}")

Sources used:
[1] - don't push in crowds - avoid placing feet on side brushes tawaf detailed instructions starting and ending tawaf : - begin from the black stone corne
[2] performing tawaf. recite surah al - kaafiroon in first rak'aa and surah al - ikhlaas in second. drink zamzam water after tawaf as the prophet did. sa'
[3] but difficult during crowded times - continue supplicating allah throughout tawaf - touch the yemeni corner if possible when reached - end each circui
[4] before leaving makkah, in compliance with the prophet's command for pilgrims to make tawaf their last engagement with the ka'ba. exceptions : - those 
[5] and is also called tawaf az - ziyaarah or tawaf al - hajj. one should circumambulate the ka'ba seven times. allah almighty said : " then let them comp


In [23]:
from pathlib import Path
from openai import OpenAI
import os

api_key = ""

client = OpenAI(api_key=api_key)
speech_file_path = "/kaggle/working/speech.mp3"

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input=answer,
    instructions="normal and friendly",
) as response:
    response.stream_to_file(speech_file_path)